# Telecom Domain: Call Logs Management with Delta Lake Operations

In this notebook, we will manage telecom call logs using Delta Lake operations such as Insert Overwrite, Append Rows, Merge Updates, and Copy Into.

### Step 1: Create and Replace Table

In [ ]:

# Create and replace a table with call logs using CTAS
spark.sql("""
CREATE OR REPLACE TABLE telecom.call_logs AS
SELECT * FROM csv.`/mnt/data/telecom_call_logs.csv`;
""")
    

### Step 2: Insert Overwrite

In [ ]:

# Insert overwrite with call logs
spark.sql("""
INSERT OVERWRITE telecom.call_logs
SELECT * FROM csv.`/mnt/data/telecom_call_logs.csv`;
""")
    

### Step 3: Append Rows

In [ ]:

# Append new rows to the telecom call logs
spark.sql("""
INSERT INTO telecom.call_logs
SELECT * FROM csv.`/mnt/data/telecom_call_logs_incremental.csv`;
""")
    

### Step 4: Merge Updates

In [ ]:

# Merge updates and new records into the telecom call logs
# Create a temporary view for the updates
spark.sql("""
CREATE OR REPLACE TEMP VIEW call_logs_updates AS
SELECT * FROM csv.`/mnt/data/telecom_call_logs_incremental.csv`;
""")
    
# Perform a merge operation
spark.sql("""
MERGE INTO telecom.call_logs a
USING call_logs_updates b
ON a.Call_ID = b.Call_ID
WHEN MATCHED THEN UPDATE SET a.Call_Duration = b.Call_Duration, a.Call_Time = b.Call_Time
WHEN NOT MATCHED THEN INSERT (Call_ID, Caller_ID, Recipient_ID, Call_Duration, Call_Time) VALUES (b.Call_ID, b.Caller_ID, b.Recipient_ID, b.Call_Duration, b.Call_Time);
""")
    

### Step 5: Copy Into for Incremental Load

In [ ]:

# Copy into the sales table for incremental loading
spark.sql("""
COPY INTO telecom.call_logs
FROM '/mnt/data/telecom_call_logs_incremental.csv'
FILEFORMAT = CSV
""")
    

### Step 6: Cleanup

In [ ]:

# Cleanup: Drop the schema and all tables within it
spark.sql("""DROP SCHEMA telecom CASCADE;""")
    